In [1]:
using Pkg
Pkg.activate("/Users/ryansenne/Documents/GitHub/SSM")
# Pkg.activate("/home/ryansenne/PycharmProjects/ssm_julia/")

  Activating project at `\Users\ryansenne\Documents\GitHub\SSM`


In [2]:
include("/Users/ryansenne/Documents/Github/SSM//src/SSM.jl")
# include("/home/ryansenne/PycharmProjects/ssm_julia/src/SSM.jl")
using LinearAlgebra
using Plots
using Distributions
using Random
using .SSM
using StatsBase

In [3]:
# generate data from a Gaussian HMM model
Random.seed!(1234)

A = [0.7 0.2 0.1; 0.1 0.7 0.2; 0.2 0.1 0.7]
    means = [[0.0, 0.0], [-1.0, 2.0], [3.0, 2.5]]
    covs = [
                [0.1 0.0; 0.0 0.1],  # Covariance matrix for state 1
                [0.1 0.0; 0.0 0.1],  # Covariance matrix for state 2
                [0.1 0.0; 0.0 0.1]   # Covariance matrix for state 3
            ]
emissions_models = [GaussianEmission(mean, cov) for (mean, cov) in zip(means, covs)]
simul_hmm = GaussianHMM(A, emissions_models, [0.33, 0.33, 0.34], 3, 2)
states, observations = SSM.sample(simul_hmm, 10000)


model = GaussianHMM(observations, 3)
baumWelch!(model, observations, 100)

-19278.766149003342
-14968.827731993188
-13272.981079080597
-13271.666647279293
-13271.666647166907


In [16]:
using DataFrames
using CSV
CSV.write("hmm_observations.csv", DataFrame(observations, :auto), writeheader=false)

"hmm_observations.csv"

In [3]:
using CSV
using DataFrames

# import old faithful data
old_faithful = CSV.read("/Users/ryansenne/PycharmProjects/NV_States_GLM_HMM/HMM_Bishop/faithful.csv", DataFrame)

# set up a variable for the HMM
X = Matrix{Float64}(old_faithful[:, [:waiting, :eruptions]])
#  zscore the data
X = (X .- mean(X, dims=1)) ./ std(X, dims=1)

# Set up initial parameters
A = [0.2 0.8; 0.5 0.5];
Pi = [0.5, 0.5];

U1 = [3, -5];
U2 = [-3, -3];
Sigma1 = [1 0; 0 1];
Sigma2 = [0.5 0; 0 0.5];

hmm = SSM.GaussianHMM(A, [GaussianEmission(U1, Sigma1), GaussianEmission(U2, Sigma2)], Pi, 2, 2)
# hmm = SSM.GaussianHMM(X, 2)
baumWelch!(hmm, X, 1)


-4584.525829637301
